# 1

### Dataset Load

In [1]:
import tensorflow as tf
import os
import cv2
import tqdm
import matplotlib.pyplot as plt

In [2]:
def parse_data(filename, label):
    img_read = tf.io.read_file(filename)
    img_decode = tf.image.decode_jpeg(img_read, channels=3)
    img_resize = tf.image.resize(img_decode, [224,224])
    return img_resize, label

def create_dataset(filenames, labels) :
    BATCH_SIZE = 16
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(16)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset    

In [22]:
names = []
labels = []
with open('./casia/label/label.txt', 'r') as f :
    lines = f.readlines()
    for line in lines :
        line = line.strip()
        names.append(line.split()[0])
        labels.append([int(i) for i in line.split()[1:]])
    
image_path = './casia/img'    
image_names = [os.path.join(image_path, name) for name in names ]

dataset = create_dataset(image_names, labels)

### Model build

In [27]:
from keras import Sequential, layers

model = Sequential()
model.add(layers.Input(shape=(224,224,3)))
model.add(layers.Conv2D(32,3,padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64,3,padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64,3,padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(128,3,padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64,3,padding='same', activation='relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(6, activation='sigmoid'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 56, 56, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 128)      

### define metrics and callbacks

In [28]:
metrics = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5),
    tf.keras.callbacks.ModelCheckpoint('./check/cp.ckpt',
                                        save_weights_only=True,
                                      save_freq='epoch'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',
                                        factor=0.1, patience=5,
                                        min_lr=1e-13)
]

### model compile

In [29]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=metrics)

### model training

In [30]:
history = model.fit(dataset,
                    batch_size=16,
                    epochs=3, 
                    callbacks=callbacks)

Epoch 1/3
625/625 [==============================] - 397s 634ms/step - loss: 0.4389 - accuracy: 0.8066 - precision: 0.6447 - recall: 0.4599 - AUC: 0.8283
Epoch 2/3
625/625 [==============================] - 482s 771ms/step - loss: 0.3757 - accuracy: 0.8364 - precision: 0.7133 - recall: 0.5495 - AUC: 0.8661
Epoch 3/3
625/625 [==============================] - 463s 740ms/step - loss: 0.3575 - accuracy: 0.8460 - precision: 0.7302 - recall: 0.5840 - AUC: 0.8795


In [33]:
resnet50 = tf.keras.applications.resnet.ResNet50(input_shape=(256,256,3),
                                                 weights='imagenet',
                                                 include_top=False)
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

# 2

### dataload - non augmentation

In [1]:
def parse_data(filename, label) :
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [256,256])
    return img, label

def create_dataset(filenames, labels) :
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(16)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

### dataload - augmentation

In [7]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import os
import cv2

names = []
labels = []
with open('./casia/label/label.txt', 'r') as f :
    lines = f.readlines()
    for line in lines :
        line = line.strip()
        names.append(line.split()[0])
        labels.append([int(i) for i in line.split()[1:]])

image_path = './casia/img'
image_list = [os.path.join(image_path,name) for name in names]

def create_img_dataset(image_list) :
    imgs = []
    for image in tqdm(image_list) :
        img = cv2.imread(image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, dsize=(256,256))
        imgs.append(img)
        
    return np.array(imgs)

image_dataset = create_img_dataset(image_list)
labels = np.array(labels)

100%|███████████████████████████████████████████████████████████████████████| 10000/10000 [00:15<00:00, 645.99it/s]


In [8]:
print('image dataset :',image_dataset.shape)
print('label dataset :',labels.shape)

image dataset : (10000, 256, 256, 3)
label dataset : (10000, 6)


In [9]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.3)

### Model build

In [10]:
from keras import Sequential, layers, activations

model = Sequential()
model.add(layers.Conv2D(64, 3, padding='same', input_shape=(256,256,3)))
model.add(layers.LeakyReLU(alpha=0.01))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(128, 3, padding='same'))
model.add(layers.LeakyReLU(alpha=0.01))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(128, 3, padding='same'))
model.add(layers.LeakyReLU(alpha=0.01))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(64, 3, padding='same'))
model.add(layers.LeakyReLU(alpha=0.01))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(6, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 128)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       1

### define metrics

In [11]:
metrics =[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]

### define callbacks

In [12]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3),
    tf.keras.callbacks.ModelCheckpoint('./ckpt_test/cp.ckpt',
                                      save_weights_only=True,
                                      save_freq='epoch'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',
                                        patience=3,
                                        factor=0.01,
                                        min_lr=1e-13)
]

### model compile

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=metrics
)

In [18]:
EPOCHS = 2
STEPS_PER_EPOCH=len(image_dataset)/16

for epoch in range(EPOCHS) :
    print()
    print()
    print("Epoch :",epoch)
    print()
    print()
    
    batch = 0
    for (batch_img, batch_label), (val_img, val_label) in zip(datagen.flow(image_dataset, labels, batch_size=16, subset='training'),
               datagen.flow(image_dataset, labels, batch_size=8, subset='validation')):
            history = model.fit(batch_img, batch_label, 
                                batch_size=16,
                                validation_data=(val_img, val_label), 
                                validation_batch_size=8,
                                verbose=0)
            print(history.history.keys())
            acc = history.history['accuracy']
            loss = history.history['loss']
#             val_acc = history.history['val_acc']
#             val_loss = history.history['val_loss']
            precision = history.history['precision']
#             val_precision = history.history['val_precision']
            recall = history.history['recall']
#             val_recall = history.history['val_recall']
            AUC = history.history['AUC']
#             val_AUC = history.history['val_AUC']
            print("Batch : {} Acc : {} Loss : {} Precision : {} Recall: {} AUC : {}".format(batch ,acc, loss, precision, recall, AUC))
            batch += 1
            
            if batch >= STEPS_PER_EPOCH :
                break
        



Epoch : 0


dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 0 Acc : [0.8125] Loss : [0.4360867738723755] Precision : [0.7058823704719543] Recall: [0.47999998927116394] AUC : [0.828169047832489]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 1 Acc : [0.8541666269302368] Loss : [0.3832630515098572] Precision : [0.7777777910232544] Recall: [0.5833333134651184] AUC : [0.8871527910232544]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 2 Acc : [0.8125] Loss : [0.40103262662887573] Precision : [0.6315789222717285] Recall: [0.52173912525177] AUC : [0.8442525863647461]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 3 Acc : [0.75

dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 30 Acc : [0.8020833730697632] Loss : [0.40864792466163635] Precision : [0.625] Recall: [0.43478259444236755] AUC : [0.8293627500534058]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 31 Acc : [0.8229166865348816] Loss : [0.40232133865356445] Precision : [0.75] Recall: [0.47999998927116394] AUC : [0.8526760339736938]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 32 Acc : [0.84375] Loss : [0.42153897881507874] Precision : [0.6875] Recall: [0.523809552192688] AUC : [0.800000011920929]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 33 Acc : [0.8229166269302368] Loss : [0.371721

dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 60 Acc : [0.7916666865348816] Loss : [0.4062547981739044] Precision : [0.6499999761581421] Recall: [0.5] AUC : [0.8686813116073608]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 61 Acc : [0.7083333730697632] Loss : [0.5813919305801392] Precision : [0.37931033968925476] Recall: [0.523809552192688] AUC : [0.7841269969940186]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 62 Acc : [0.7708333134651184] Loss : [0.49788206815719604] Precision : [0.5625] Recall: [0.375] AUC : [0.7728587985038757]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 63 Acc : [0.8020833730697632] Loss : [

dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 90 Acc : [0.7083333730697632] Loss : [0.4776347875595093] Precision : [0.375] Recall: [0.25] AUC : [0.7734375]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 91 Acc : [0.7395833730697632] Loss : [0.44451624155044556] Precision : [0.4117647111415863] Recall: [0.3181818127632141] AUC : [0.7991400957107544]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 92 Acc : [0.8020833730697632] Loss : [0.44982507824897766] Precision : [0.6875] Recall: [0.4399999976158142] AUC : [0.7952112555503845]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 93 Acc : [0.8333333730697632] Loss : [0.34342

dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 121 Acc : [0.8541666865348816] Loss : [0.29377657175064087] Precision : [0.625] Recall: [0.5555555820465088] AUC : [0.9077635407447815]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 122 Acc : [0.8020833730697632] Loss : [0.4149171710014343] Precision : [0.5625] Recall: [0.4285714328289032] AUC : [0.8114286065101624]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 123 Acc : [0.78125] Loss : [0.47038280963897705] Precision : [0.6000000238418579] Recall: [0.375] AUC : [0.7960069179534912]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 124 Acc : [0.8020833134651184] Loss : [0.40

dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 152 Acc : [0.7604166865348816] Loss : [0.48995304107666016] Precision : [0.625] Recall: [0.37037035822868347] AUC : [0.773751974105835]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 153 Acc : [0.78125] Loss : [0.40426158905029297] Precision : [0.5] Recall: [0.380952388048172] AUC : [0.8396825194358826]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 154 Acc : [0.7604166269302368] Loss : [0.45699629187583923] Precision : [0.5625] Recall: [0.36000001430511475] AUC : [0.7923943996429443]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 155 Acc : [0.78125] Loss : [0.45638656616210

dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 182 Acc : [0.7708333730697632] Loss : [0.44846609234809875] Precision : [0.5625] Recall: [0.375] AUC : [0.8162615299224854]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 183 Acc : [0.7708333730697632] Loss : [0.5405809283256531] Precision : [0.6875] Recall: [0.3928571343421936] AUC : [0.7602416276931763]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 184 Acc : [0.8125] Loss : [0.3771539032459259] Precision : [0.5625] Recall: [0.44999998807907104] AUC : [0.8641446828842163]
dict_keys(['loss', 'accuracy', 'precision', 'recall', 'AUC', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC'])
Batch : 185 Acc : [0.8125] Loss : [0.3835486173629761] Precisio

KeyboardInterrupt: 

In [79]:
from keras import Sequential, layers, activations

model2 = Sequential()
model2.add(layers.Conv2D(64, 3, padding='same', input_shape=(256,256,3)))
model2.add(layers.LeakyReLU(alpha=0.01))
model2.add(layers.MaxPool2D())

model2.add(layers.Conv2D(128, 3, padding='same'))
model2.add(layers.LeakyReLU(alpha=0.01))
model2.add(layers.MaxPool2D())

model2.add(layers.Conv2D(128, 3, padding='same'))
model2.add(layers.LeakyReLU(alpha=0.01))
model2.add(layers.MaxPool2D())

model2.add(layers.Conv2D(64, 3, padding='same'))
model2.add(layers.LeakyReLU(alpha=0.01))

model2.add(layers.GlobalAveragePooling2D())
model2.add(layers.Dense(10, activation='sigmoid'))

model2.summary()
metrics =[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC')
]
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3),
    tf.keras.callbacks.ModelCheckpoint('./ckpt_test/cp.ckpt',
                                      save_weights_only=True,
                                      save_freq='epoch'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy',
                                        patience=3,
                                        factor=0.01,
                                        min_lr=1e-13)
]
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=metrics
)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 128, 128, 128)     0         
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 64, 64, 128)     

In [86]:
num_classes = 10
epochs = 2

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)


for e in range(epochs) :
    print('Epochs : ',e)
    batches = 0
    for (x_batch, y_batch), (val_x, val_y) in zip(_datagen.flow(x_train, y_train, batch_size=32, subset='training'),
                               _datagen.flow(x_train, y_train, batch_size=8, subset='validation')) :
        print(batches)
        model2.fit(x_batch, y_batch,
                   validation_data=(val_x, val_y),
                   verbose=1)
        batches += 1
        
        if batches >= len(x_train) / 32:
            break


# # compute quantities required for featurewise normalization
# # (std, mean, and principal components if ZCA whitening is applied)
# _datagen.fit(x_train)

# # fits the model on batches with real-time data augmentation:
# model2.fit(_datagen.flow(x_train, y_train, batch_size=32,
#          subset='training'),
#          validation_data=_datagen.flow(x_train, y_train,
#          batch_size=8, subset='validation'),
#          steps_per_epoch=len(x_train) / 32, epochs=epochs)

Epochs :  0
0
1/1 [==============================] - 0s 345ms/step - loss: 0.1765 - accuracy: 0.9344 - precision: 0.7619 - recall: 0.5000 - AUC: 0.9181 - val_loss: 0.1600 - val_accuracy: 0.9250 - val_precision: 0.7500 - val_recall: 0.3750 - val_AUC: 0.9661
1
1/1 [==============================] - 0s 38ms/step - loss: 0.1478 - accuracy: 0.9406 - precision: 0.9333 - recall: 0.4375 - AUC: 0.9547 - val_loss: 0.1556 - val_accuracy: 0.9250 - val_precision: 1.0000 - val_recall: 0.2500 - val_AUC: 0.9575
2
1/1 [==============================] - 0s 38ms/step - loss: 0.1974 - accuracy: 0.9312 - precision: 0.7778 - recall: 0.4375 - AUC: 0.9091 - val_loss: 0.1820 - val_accuracy: 0.9375 - val_precision: 1.0000 - val_recall: 0.3750 - val_AUC: 0.9201
3
1/1 [==============================] - 0s 43ms/step - loss: 0.1827 - accuracy: 0.9406 - precision: 0.7826 - recall: 0.5625 - AUC: 0.9107 - val_loss: 0.1438 - val_accuracy: 0.9625 - val_precision: 1.0000 - val_recall: 0.6250 - val_AUC: 0.9688
4
1/1 [====

32
1/1 [==============================] - 0s 59ms/step - loss: 0.1765 - accuracy: 0.9375 - precision: 0.8750 - recall: 0.4375 - AUC: 0.9262 - val_loss: 0.2286 - val_accuracy: 0.8875 - val_precision: 0.4000 - val_recall: 0.2500 - val_AUC: 0.8741
33
1/1 [==============================] - 0s 60ms/step - loss: 0.1818 - accuracy: 0.9281 - precision: 0.8462 - recall: 0.3438 - AUC: 0.9352 - val_loss: 0.1978 - val_accuracy: 0.9375 - val_precision: 0.8000 - val_recall: 0.5000 - val_AUC: 0.8845
34
1/1 [==============================] - 0s 69ms/step - loss: 0.2001 - accuracy: 0.9187 - precision: 0.7500 - recall: 0.2812 - AUC: 0.9026 - val_loss: 0.2750 - val_accuracy: 0.9000 - val_precision: 0.5000 - val_recall: 0.2500 - val_AUC: 0.8099
35
1/1 [==============================] - 0s 55ms/step - loss: 0.1616 - accuracy: 0.9312 - precision: 0.8125 - recall: 0.4062 - AUC: 0.9454 - val_loss: 0.1510 - val_accuracy: 0.9500 - val_precision: 0.8333 - val_recall: 0.6250 - val_AUC: 0.9540
36
1/1 [============

1/1 [==============================] - 0s 57ms/step - loss: 0.1379 - accuracy: 0.9406 - precision: 0.8824 - recall: 0.4688 - AUC: 0.9652 - val_loss: 0.1784 - val_accuracy: 0.9375 - val_precision: 0.8000 - val_recall: 0.5000 - val_AUC: 0.9297
99
1/1 [==============================] - 0s 60ms/step - loss: 0.2667 - accuracy: 0.9062 - precision: 0.5714 - recall: 0.2500 - AUC: 0.8370 - val_loss: 0.2260 - val_accuracy: 0.9375 - val_precision: 1.0000 - val_recall: 0.3750 - val_AUC: 0.8681
100
1/1 [==============================] - 0s 62ms/step - loss: 0.1854 - accuracy: 0.9406 - precision: 0.9333 - recall: 0.4375 - AUC: 0.9193 - val_loss: 0.1770 - val_accuracy: 0.9250 - val_precision: 0.6250 - val_recall: 0.6250 - val_AUC: 0.9418
101
1/1 [==============================] - 0s 54ms/step - loss: 0.1822 - accuracy: 0.9344 - precision: 0.8667 - recall: 0.4062 - AUC: 0.9121 - val_loss: 0.2420 - val_accuracy: 0.9000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_AUC: 0.8689
102
1/1 [====

1/1 [==============================] - 0s 56ms/step - loss: 0.1858 - accuracy: 0.9250 - precision: 0.8333 - recall: 0.3125 - AUC: 0.9153 - val_loss: 0.1577 - val_accuracy: 0.9375 - val_precision: 0.8000 - val_recall: 0.5000 - val_AUC: 0.9479
165
1/1 [==============================] - 0s 64ms/step - loss: 0.1361 - accuracy: 0.9469 - precision: 0.8571 - recall: 0.5625 - AUC: 0.9712 - val_loss: 0.1653 - val_accuracy: 0.9250 - val_precision: 0.7500 - val_recall: 0.3750 - val_AUC: 0.9497
166
1/1 [==============================] - 0s 61ms/step - loss: 0.1429 - accuracy: 0.9438 - precision: 0.8500 - recall: 0.5312 - AUC: 0.9618 - val_loss: 0.2085 - val_accuracy: 0.9125 - val_precision: 1.0000 - val_recall: 0.1250 - val_AUC: 0.9045
167
1/1 [==============================] - 0s 60ms/step - loss: 0.1483 - accuracy: 0.9375 - precision: 0.8000 - recall: 0.5000 - AUC: 0.9525 - val_loss: 0.1223 - val_accuracy: 0.9750 - val_precision: 1.0000 - val_recall: 0.7500 - val_AUC: 0.9635
168
1/1 [===========

1/1 [==============================] - 0s 93ms/step - loss: 0.1990 - accuracy: 0.9250 - precision: 0.7500 - recall: 0.3750 - AUC: 0.9071 - val_loss: 0.1271 - val_accuracy: 0.9625 - val_precision: 1.0000 - val_recall: 0.6250 - val_AUC: 0.9792
231
1/1 [==============================] - 0s 140ms/step - loss: 0.1755 - accuracy: 0.9312 - precision: 0.7500 - recall: 0.4688 - AUC: 0.9147 - val_loss: 0.2392 - val_accuracy: 0.9125 - val_precision: 1.0000 - val_recall: 0.1250 - val_AUC: 0.8455
232
1/1 [==============================] - 0s 72ms/step - loss: 0.2064 - accuracy: 0.9094 - precision: 0.5882 - recall: 0.3125 - AUC: 0.8898 - val_loss: 0.2046 - val_accuracy: 0.9250 - val_precision: 0.7500 - val_recall: 0.3750 - val_AUC: 0.8950
233
1/1 [==============================] - 0s 64ms/step - loss: 0.1977 - accuracy: 0.9219 - precision: 0.7333 - recall: 0.3438 - AUC: 0.9064 - val_loss: 0.1090 - val_accuracy: 0.9500 - val_precision: 0.8333 - val_recall: 0.6250 - val_AUC: 0.9844
234
1/1 [==========

...

1/1 [==============================] - 0s 80ms/step - loss: 0.2012 - accuracy: 0.9281 - precision: 0.6957 - recall: 0.5000 - AUC: 0.9003 - val_loss: 0.2209 - val_accuracy: 0.9500 - val_precision: 1.0000 - val_recall: 0.5000 - val_AUC: 0.8568
363
1/1 [==============================] - 0s 99ms/step - loss: 0.1351 - accuracy: 0.9469 - precision: 0.8000 - recall: 0.6250 - AUC: 0.9633 - val_loss: 0.1299 - val_accuracy: 0.9625 - val_precision: 0.8571 - val_recall: 0.7500 - val_AUC: 0.9592
364
1/1 [==============================] - 0s 88ms/step - loss: 0.1521 - accuracy: 0.9375 - precision: 0.8750 - recall: 0.4375 - AUC: 0.9495 - val_loss: 0.2022 - val_accuracy: 0.9000 - val_precision: 0.5000 - val_recall: 0.2500 - val_AUC: 0.9062
365
1/1 [==============================] - 0s 100ms/step - loss: 0.2061 - accuracy: 0.9344 - precision: 0.7895 - recall: 0.4688 - AUC: 0.8918 - val_loss: 0.1978 - val_accuracy: 0.9125 - val_precision: 0.6667 - val_recall: 0.2500 - val_AUC: 0.9132
366
1/1 [==========

1/1 [==============================] - 0s 57ms/step - loss: 0.1911 - accuracy: 0.9281 - precision: 0.7368 - recall: 0.4375 - AUC: 0.9168 - val_loss: 0.1188 - val_accuracy: 0.9500 - val_precision: 0.8333 - val_recall: 0.6250 - val_AUC: 0.9852
429
1/1 [==============================] - 0s 60ms/step - loss: 0.1833 - accuracy: 0.9312 - precision: 0.7083 - recall: 0.5312 - AUC: 0.9348 - val_loss: 0.1847 - val_accuracy: 0.9375 - val_precision: 0.8000 - val_recall: 0.5000 - val_AUC: 0.9227
430
1/1 [==============================] - 0s 58ms/step - loss: 0.1659 - accuracy: 0.9281 - precision: 0.8000 - recall: 0.3750 - AUC: 0.9530 - val_loss: 0.1787 - val_accuracy: 0.9625 - val_precision: 1.0000 - val_recall: 0.6250 - val_AUC: 0.9123
431
1/1 [==============================] - 0s 54ms/step - loss: 0.1706 - accuracy: 0.9469 - precision: 0.8261 - recall: 0.5938 - AUC: 0.9293 - val_loss: 0.2059 - val_accuracy: 0.9125 - val_precision: 0.6000 - val_recall: 0.3750 - val_AUC: 0.9193
432
1/1 [===========

1/1 [==============================] - 0s 57ms/step - loss: 0.1709 - accuracy: 0.9406 - precision: 0.8095 - recall: 0.5312 - AUC: 0.9321 - val_loss: 0.1458 - val_accuracy: 0.9375 - val_precision: 0.7143 - val_recall: 0.6250 - val_AUC: 0.9705
495
1/1 [==============================] - 0s 66ms/step - loss: 0.2731 - accuracy: 0.9062 - precision: 0.5500 - recall: 0.3438 - AUC: 0.8727 - val_loss: 0.1115 - val_accuracy: 0.9625 - val_precision: 1.0000 - val_recall: 0.6250 - val_AUC: 0.9722
496
1/1 [==============================] - 0s 63ms/step - loss: 0.1756 - accuracy: 0.9219 - precision: 0.6522 - recall: 0.4688 - AUC: 0.9397 - val_loss: 0.2422 - val_accuracy: 0.9125 - val_precision: 0.6000 - val_recall: 0.3750 - val_AUC: 0.9106
497
1/1 [==============================] - 0s 58ms/step - loss: 0.1644 - accuracy: 0.9344 - precision: 0.7895 - recall: 0.4688 - AUC: 0.9429 - val_loss: 0.0748 - val_accuracy: 0.9875 - val_precision: 0.8889 - val_recall: 1.0000 - val_AUC: 0.9913
498
1/1 [===========

1/1 [==============================] - 0s 61ms/step - loss: 0.1745 - accuracy: 0.9312 - precision: 0.7500 - recall: 0.4688 - AUC: 0.9376 - val_loss: 0.1141 - val_accuracy: 0.9750 - val_precision: 1.0000 - val_recall: 0.7500 - val_AUC: 0.9913
561
1/1 [==============================] - 0s 59ms/step - loss: 0.1827 - accuracy: 0.9312 - precision: 0.7273 - recall: 0.5000 - AUC: 0.9281 - val_loss: 0.1440 - val_accuracy: 0.9375 - val_precision: 1.0000 - val_recall: 0.3750 - val_AUC: 0.9653
562
1/1 [==============================] - 0s 62ms/step - loss: 0.1725 - accuracy: 0.9312 - precision: 0.7500 - recall: 0.4688 - AUC: 0.9255 - val_loss: 0.1751 - val_accuracy: 0.9375 - val_precision: 1.0000 - val_recall: 0.3750 - val_AUC: 0.9210
563
1/1 [==============================] - 0s 74ms/step - loss: 0.1442 - accuracy: 0.9344 - precision: 0.7391 - recall: 0.5312 - AUC: 0.9630 - val_loss: 0.1285 - val_accuracy: 0.9375 - val_precision: 0.8000 - val_recall: 0.5000 - val_AUC: 0.9757
564
1/1 [===========

KeyboardInterrupt: 